# Import required packages

In [1]:
import csv
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import InputLayer, Dropout, Dense 
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD

# Specify path of weights file and data

In [2]:
path = os.getcwd()
dataset_path = os.path.join(path, 'keypoint_data.csv')
model_path = os.path.join(path, 'weights.hdf5')

Define labels

In [3]:
labels = ["None", "Rock", "Paper", "Scissors", "Like", "Dislike"]


num_classes = len(labels)

Read dataset

In [4]:
dataset = pd.read_csv(dataset_path, header = None)

In [5]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,2.0,0.0,0.0,0.178462,-0.095385,0.323077,-0.243077,0.436923,-0.353846,0.547692,...,-0.083077,-0.935385,-0.138462,-0.409231,-0.193846,-0.563077,-0.224615,-0.664615,-0.249231,-0.763077
1,2.0,0.0,0.0,0.171340,-0.093458,0.314642,-0.246106,0.433022,-0.355140,0.548287,...,-0.090343,-0.943925,-0.149533,-0.401869,-0.205607,-0.560748,-0.233645,-0.660436,-0.261682,-0.760125
2,2.0,0.0,0.0,0.175000,-0.100000,0.318750,-0.253125,0.431250,-0.365625,0.543750,...,-0.087500,-0.937500,-0.150000,-0.415625,-0.209375,-0.568750,-0.237500,-0.671875,-0.262500,-0.768750
3,2.0,0.0,0.0,0.179811,-0.091483,0.324921,-0.233438,0.438486,-0.353312,0.539432,...,-0.141956,-0.921136,-0.164038,-0.403785,-0.227129,-0.555205,-0.258675,-0.646688,-0.280757,-0.728707
4,2.0,0.0,0.0,0.164596,-0.099379,0.304348,-0.245342,0.413043,-0.360248,0.512422,...,-0.155280,-0.909938,-0.177019,-0.403727,-0.239130,-0.559006,-0.273292,-0.658385,-0.298137,-0.757764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,0.0,0.0,0.0,0.126168,-0.182243,0.135514,-0.378505,0.070093,-0.528037,-0.004673,...,-0.154206,-0.350467,-0.182243,-0.457944,-0.280374,-0.644860,-0.331776,-0.742991,-0.364486,-0.850467
1965,0.0,0.0,0.0,0.106280,-0.173913,0.101449,-0.367150,0.033816,-0.512077,-0.033816,...,-0.198068,-0.289855,-0.169082,-0.439614,-0.275362,-0.637681,-0.333333,-0.753623,-0.367150,-0.879227
1966,0.0,0.0,0.0,0.056122,-0.183673,0.010204,-0.372449,-0.096939,-0.510204,-0.209184,...,-0.285714,-0.290816,-0.147959,-0.500000,-0.275510,-0.719388,-0.336735,-0.846939,-0.377551,-0.979592
1967,0.0,0.0,0.0,0.020513,-0.179487,-0.020513,-0.364103,-0.117949,-0.502564,-0.215385,...,-0.271795,-0.287179,-0.128205,-0.512821,-0.251282,-0.728205,-0.323077,-0.861538,-0.364103,-1.000000


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 43 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1969 non-null   float64
 1   1       1969 non-null   float64
 2   2       1969 non-null   float64
 3   3       1969 non-null   float64
 4   4       1969 non-null   float64
 5   5       1969 non-null   float64
 6   6       1969 non-null   float64
 7   7       1969 non-null   float64
 8   8       1969 non-null   float64
 9   9       1969 non-null   float64
 10  10      1969 non-null   float64
 11  11      1969 non-null   float64
 12  12      1969 non-null   float64
 13  13      1969 non-null   float64
 14  14      1969 non-null   float64
 15  15      1969 non-null   float64
 16  16      1969 non-null   float64
 17  17      1969 non-null   float64
 18  18      1969 non-null   float64
 19  19      1969 non-null   float64
 20  20      1969 non-null   float64
 21  21      1969 non-null   float64
 22  

Split dataframe to training set and labels

In [7]:
y = np.array(dataset[0])
X = np.array(dataset.drop(columns=[0]))
# One hot encoding
y = to_categorical(y).astype('int')

Split training set to train and validation set

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size= 0.2, random_state = 42)

# Define and train model

In [9]:
model = Sequential()

model.add(InputLayer((42,)))
model.add(Dense(20, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(num_classes, activation = 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                860       
_________________________________________________________________
dropout (Dropout)            (None, 20)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 66        
Total params: 1,136
Trainable params: 1,136
Non-trainable params: 0
_________________________________________________________________


2022-10-09 21:02:12.641300: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-09 21:02:12.642601: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [10]:
# opt = SGD(learning_rate=0.1)

model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train, y_train, epochs = 1000, batch_size=128, validation_data=(X_val, y_val))

2022-10-09 21:02:24.468811: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/1000
13/13 [==============================] - 1s 26ms/step - loss: 1.8289 - accuracy: 0.2273 - val_loss: 1.7553 - val_accuracy: 0.2614
Epoch 2/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.7612 - accuracy: 0.2946 - val_loss: 1.7206 - val_accuracy: 0.3706
Epoch 3/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.7210 - accuracy: 0.3092 - val_loss: 1.6916 - val_accuracy: 0.3858
Epoch 4/1000
13/13 [==============================] - 0s 5ms/step - loss: 1.6907 - accuracy: 0.3416 - val_loss: 1.6547 - val_accuracy: 0.4010
Epoch 5/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.6544 - accuracy: 0.3721 - val_loss: 1.6105 - val_accuracy: 0.4188
Epoch 6/1000
13/13 [==============================] - 0s 8ms/step - loss: 1.6189 - accuracy: 0.3784 - val_loss: 1.5680 - val_accuracy: 0.4594
Epoch 7/1000
13/13 [==============================] - 0s 7ms/step - loss: 1.5757 - accuracy: 0.3994 - val_loss: 1.5232 - val_accuracy: 0.4645
Epoch

In [11]:
model.save(model_path)